In [ ]:
from selenium import webdriver
from pymongo import MongoClient
import re
from flask import Flask, render_template, jsonify

In [ ]:
from selenium.webdriver.common.keys import Keys

In [ ]:
import pandas as pd

In [ ]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [ ]:
pd.DataFrame(columns=["Caractéristiques(Chambres,Douches et Surfaces)","Emplacements","Prix"])

In [ ]:
service=Service('C:/Users/hechm/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')

In [ ]:
driver=webdriver.Chrome(service=service)

In [ ]:
caracteristiques_list=[]
emplacements_list=[]
prix_list=[]

In [ ]:
def extraire_donnees(url_pattern, page_range):
    caracteristiques = []
    emplacements = []
    prix = []

    for i in page_range:
        url = url_pattern.format(i)
        driver.get(url)
        
        # Extraction des caractéristiques
        caracteristiques.extend([c.text for c in driver.find_elements(By.CLASS_NAME, 'li-list-item-info-span')])
        
        # Extraction des emplacements à partir des balises <p>
        emplacements.extend([p.text for p in driver.find_elements(By.TAG_NAME, 'p') if p.text.strip() != ""])
        
        # Extraction des prix
        prix.extend([p.text for p in driver.find_elements(By.CLASS_NAME, 'item-box-price')])

    return caracteristiques, emplacements, prix

In [ ]:
page_range = range(1, 3)
appartements_url = "https://www.menzili.tn/immo/appartement-tunisie?l=0&page={}&tri=1"
maisons_url = "https://www.menzili.tn/immo/maison-tunisie?l=0&page={}&tri=1"
terrains_url = "https://www.menzili.tn/immo/terrain-tunisie?l=0&page={}&tri=1"

In [ ]:
caracteristiques_appartements, emplacements_appartements, prix_appartements = extraire_donnees(appartements_url, page_range)
caracteristiques_maisons, emplacements_maisons, prix_maisons = extraire_donnees(maisons_url, page_range)
caracteristiques_terrains, emplacements_terrains, prix_terrains = extraire_donnees(terrains_url, page_range)

In [ ]:
for a in caracteristiques_appartements:
     if a not in ["Chambres","Salle","de","bain","Piéces","(Total)","Surf","habitable","terrain"]:
        caracteristiques_list.append(a.split())
for m in caracteristiques_maisons:
     if m not in ["Chambres","Salle","de","bain","Piéces","(Total)","Surf","habitable","terrain"]:
        caracteristiques_list.append(m.split())
for t in caracteristiques_terrains:
     if t not in ["Surf","terrain"]:
        caracteristiques_list.append(t.split())

for ea in emplacements_appartements:
    emplacements_list.append(ea.strip())
for em in emplacements_maisons:
    emplacements_list.append(em.strip())
for et in emplacements_terrains:
    emplacements_list.append(et.strip())

for pa in prix_appartements:
    prix_list.append(pa.strip())
for pm in prix_maisons:
    prix_list.append(pm.strip())
for pt in prix_terrains:
    prix_list.append(pt.strip())

data_tuples=list(zip(caracteristiques_list,emplacements_list,prix_list))
pd.DataFrame(data_tuples,columns=["Caractéristiques(Chambres,Douches et Surfaces)","Emplacements","Prix"])

In [ ]:
def inserer_annonces():
    client=MongoClient('mongodb://localhost:27017/')
    annonces=[{"Caracteristiques":caracteristique,"Emplacements":emplacement,"Prix":prix} for caracteristique,emplacement,prix in zip(caracteristiques_list,emplacements_list,prix_list)]
    with client:
        db=client.immobilieres
        db.annonces.insert_many(annonces)

In [ ]:
def afficher_annonces():
    client=MongoClient('mongodb://localhost:27017/')
    with client:
        db=client.immobilieres
        annonces=db.annonces.find({},{"_id":0})
        return list(annonces)
print(afficher_annonces())

In [ ]:
def compter_annonces():
    client=MongoClient('mongodb://localhost:27017/')
    with client:
        db=client.immobilieres
        return db.annonces.count_documents({})

In [ ]:
app=Flask(__name__)
@app.route('/')
def main():
    try:
        return render_template('main.twig')
    except Exception as e:
        return jsonify({"Error":str(e)})
@app.route('/scrape')
def scrape():
    try:
        inserer_annonces()
        return jsonify({"message": "Scraping complet!"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


# Route pour afficher les données
@app.route('/data')
def data():
    try:
        annonces = afficher_annonces()
        return render_template('data.twig', annonces=annonces)
    except Exception as e:
        return jsonify({"error": str(e)}), 500


# Route pour les statistiques
@app.route('/stats')
def stats():
    try:
        count = compter_annonces()
        return jsonify({"total_listings": count})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == '__main__':
    app.run(debug=True, port=5001, use_reloader=False)

In [ ]:
driver.quit()

In [ ]:
%tb